In [1]:
!pip install -qq nest_asyncio pydantic pydantic_ai rich html2text python-dotenv

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from rich import print as pprint

In [5]:
from pydantic_ai import Agent
hello_word_agent=Agent(
    model="groq:llama-3.2-3b-preview",
    system_prompt="You are an excellent cook."
)

result=hello_word_agent.run_sync("Suggest me 3 recipe for today's breakfast")


In [9]:
pprint(result.data)

I'd be happy to suggest three delicious breakfast recipes for you to try today. Here they are:

**Recipe 1: Avocado and Bacon Omelette**

Ingredients:

* 2 eggs
* 1/2 avocado, diced
* 4 slices of bacon, cooked and crumbled
* Salt and pepper to taste
* Optional: Chopped fresh herbs like parsley or chives

Instructions:

1. Beat the eggs in a bowl and season with salt and pepper.
2. Heat a non-stick pan over medium heat and add the diced avocado.
3. Pour in the eggs and cook until the edges start to set.
4. Add the crumbled bacon and fold the omelette in half.
5. Cook for another minute and serve hot.

**Recipe 2: Cinnamon French Toast with Fresh Berries**

Ingredients:

* 4 slices of bread (preferably a day-old bread)
* 2 eggs
* 1/2 cup of milk
* 1/4 cup of granulated sugar
* 1 tsp of ground cinnamon
* 2 tbsp of unsalted butter, melted
* Fresh berries (such as strawberries, blueberries, or raspberries)

Instructions:

1. In a bowl, whisk together the eggs, milk, sugar, and cinnamon.
2. Heat a non-stick pan over medium heat and dip each bread slice into the egg mixture, coating both sides evenly.
3. Cook the French toast for 2-3 minutes on each side, or until golden brown.
4. Serve with melted butter and fresh berries.

**Recipe 3: Greek Yogurt Parfait with Granola and Banana**

Ingredients:

* 1 cup of Greek yogurt
* 1/4 cup of granola
* 1 ripe banana, sliced
* 1 tsp of honey (optional)
* Fresh mint leaves (optional)

Instructions:

1. Layer the Greek yogurt, granola, and banana slices in a bowl.
2. Drizzle with honey, if desired.
3. Garnish with fresh mint leaves, if desired.
4. Serve chilled and enjoy!

Which one of these recipes sounds appealing to you?

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import html2text
import re

def clean_text(text):
    """Clean extracted text by removing extra whitespace and empty lines"""
    text = re.sub(r'\n\s*\n', '\n\n', text.strip())
    return text

def scrape_website(url, selector=None):
    """
    Scrapes data from a website and converts HTML to Markdown.
    
    Parameters:
    url (str): The URL of the website to scrape
    selector (str, optional): CSS selector to target specific elements
    
    Returns:
    str: Markdown formatted text
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove unwanted elements
        for element in soup.select('script, style, nav, footer, header'):
            element.decompose()
        
        # Convert to Markdown
        h = html2text.HTML2Text()
        h.ignore_links = False
        h.ignore_images = False
        h.body_width = 0  # No wrapping
        
        if selector:
            elements = soup.select(selector)
            content = '\n\n'.join(h.handle(str(element)) for element in elements)
        else:
            content = h.handle(str(soup.body))
        
        # Clean and save content
        content = clean_text(content)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        with open(f'scraped_content_{timestamp}.md', 'w', encoding='utf-8') as file:
            file.write(content)
            
        return content
        
    except Exception as e:
        print(f"Error: {e}")
        return ""



In [ ]:

url=web address to job posting

What is agent suppose to do?

1.Fetch the job data from the posting URL
2. Use the job posting and write the cold email


In [13]:
from dataclasses import dataclass
from typing import List

from pydantic import BaseModel, Field
from pydantic_ai import RunContext

@dataclass
class JobInformationFetchDeps:
    job_post_url:str

@dataclass
class JobDescriptionAgentDependencies:
    job_posting_information:str


class JobDescriptionAgentResult(BaseModel):
    role: str = Field(
        description="The job title or role position being described (e.g., 'Senior Software Engineer', 'Product Manager')"
    )
    company_name: str = Field(description="The Company which posted job")
    experience: str = Field(
        description="Required years and type of experience for the position (e.g., '5+ years of software development')"
    )
    skills: List[str] = Field(
        description="List of specific technical skills, tools, or competencies required for the role (e.g., ['Python', 'AWS', 'Machine Learning'])"
    )
    description: str = Field(
        description="Detailed overview of the job responsibilities, requirements, and expectations"
    )

job_description_parser_agent = Agent(
    model="groq:llama-3.2-3b-preview",
    deps_type=JobInformationFetchDeps,
    result_type=JobDescriptionAgentResult,
    system_prompt="""You are a specialized HR assistant focused on analyzing and structuring job descriptions. Your primary responsibilities are:
1. Use the get_job_details tool to retrieve job posting information
2. Extract and categorize key components including:
   - Core role/position title
   - Required experience level
   - Essential skills and qualifications
   - Detailed role description and responsibilities
Format all outputs according to the JobDescription schema. Be precise and consistent in your categorization. When analyzing skills:
- Focus on specific technical and professional competencies
- Separate distinct skills into individual items
- Standardize skill names (e.g., "Python" not "python programming")
If job details are ambiguous or incomplete, make reasonable inferences based on industry standards while maintaining accuracy.""",
)

@job_description_parser_agent.tool
def get_job_details(
    ctx: RunContext[JobInformationFetchDeps],
) -> JobDescriptionAgentDependencies:
    """
    Retrieves and extracts job posting information
    """
    job_post_url = ctx.deps.job_post_url
    job_posting_information =scrape_website(url=job_post_url)
    
    return JobDescriptionAgentDependencies(
        job_posting_information=job_posting_information
    )




In [ ]:
job_description_agent_result = job_description_parser_agent.run_sync(
    "Please extract job description for the provided URL",
    deps=JobInformationFetchDeps(
        job_post_url="https://boards.greenhouse.io/anthropic/jobs/4042814008"
    ),
)
pprint(job_description_agent_result.data)

In [15]:
class ColdEmailWriterAgentInput(BaseModel):
    job_description: JobDescriptionAgentResult = Field(
        description="Parsed job posting details including role, company, required experience, skills, and full description"
    )

class ColdEmailWriterAgentResponse(BaseModel):
    subject: str = Field(
        description="Email subject line that captures attention and highlights key value proposition (e.g., 'Experienced Python/ML Engineers Available for Anthropic's AI Initiative')"
    )
    body: str = Field(
        description="Professional email body that matches job requirements with portfolio expertise, includes introduction, value proposition, relevant project examples, and call-to-action"
    )

cold_email_writer_agent=Agent(
    model='groq:llama-3.2-3b-preview',
    deps_type=ColdEmailWriterAgentInput,
    result_type=ColdEmailWriterAgentResponse,
    system_prompt="""You are Ria, a tech recruitment specialist at Turing, reaching out to hiring managers about your firm's pre-vetted engineering talent pool. Using the provided job description:

1. Analyze role requirements and highlight relevant Turing portfolio projects
2. Create concise, compelling subject lines highlighting available talent
3. Write brief, impactful email body (3-4 paragraphs max) that:
   - Opens with specific reference to company's hiring needs
   - Showcases relevant Turing portfolio projects matching required tech stack
   - Emphasizes that Turing has pre-vetted engineers ready to interview
   - Includes clear call-to-action to discuss available candidates

Keep tone professional yet conversational. Focus on Turing's talent pool and proven project experience.
"""
)

In [16]:
result=cold_email_writer_agent.run_sync(
    "Please write a cold email",
    deps=ColdEmailWriterAgentInput(
        job_description=job_description_agent_result.data
    )
)
pprint(result.data)

ColdEmailWriterAgentResponse(
    subject='Unlock Expertise for Your AI Initiative',
    body="Dear Hiring Manager,\n\nI came across your job posting for an AI Engineer and was impressed by the 
innovative projects you're working on. At Turing, we've helped numerous companies like yours find top talent with 
expertise in AI and machine learning.\n\nOur portfolio showcases projects that align with your requirements, 
including a natural language processing project that improved sentiment analysis by 30% and a computer vision 
project that increased object detection accuracy by 25%. Our engineers have hands-on experience with popular 
frameworks like TensorFlow and PyTorch.\n\nWhat sets us apart is our pre-vetted talent pool, carefully selected and
matched to your specific needs. Our engineers are ready to interview and contribute to your team's success.\n\nIf 
you're interested in exploring our available candidates, I'd love to schedule a call to discuss further. Please let
me know if you'd like to learn more about our expertise and how we can support your AI initiative.\n\nBest regards,
Ria"
)

In [17]:
pprint(result.data.body)

Dear Hiring Manager,

I came across your job posting for an AI Engineer and was impressed by the innovative projects you're working on. 
At Turing, we've helped numerous companies like yours find top talent with expertise in AI and machine learning.

Our portfolio showcases projects that align with your requirements, including a natural language processing project
that improved sentiment analysis by 30% and a computer vision project that increased object detection accuracy by 
25%. Our engineers have hands-on experience with popular frameworks like TensorFlow and PyTorch.

What sets us apart is our pre-vetted talent pool, carefully selected and matched to your specific needs. Our 
engineers are ready to interview and contribute to your team's success.

If you're interested in exploring our available candidates, I'd love to schedule a call to discuss further. Please 
let me know if you'd like to learn more about our expertise and how we can support your AI initiative.

Best regards, Ria